In [2]:
# Setup environment
%run 0-Environment_Setup.ipynb

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
Stored 's3_datalake_path_csv' (str)
Stored 'local_data_path_csv' (str)
Stored 's3_datalake_path_parquet' (str)


In [3]:
!pip install -U sagemaker

In [4]:
import sys

import boto3
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession

In [5]:
# Set session variables
sm_client = boto3.client('sagemaker', region_name=region)
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sagemaker_session.boto_session.region_name
bucket = sess.default_bucket()
pipeline_session = PipelineSession()

In [70]:
base_path = f"s3://{bucket}/store-sales-forecasting/pipelines"
input_data_path = f"{base_path}/input/"
output_data_path = f"{base_path}/output/"
print(input_data_path)
print(output_data_path)

s3://sagemaker-us-east-1-342408968837/store-sales-forecasting/pipelines/input/
s3://sagemaker-us-east-1-342408968837/store-sales-forecasting/pipelines/output/


In [294]:
# Pull the data from the feature store sorted by date and then store number
sales_features_store_df = get_store_dataset_from_offline_feature_group_date_sort(store_sales_feature_group)
sales_features_store_df.head()

Running 
    SELECT *
    FROM
        "store_sales_feature_group_offline_1728878780"
    ORDER BY
        date ASC, store_nbr ASC
    


INFO:sagemaker:Query 37b3fff8-a838-483f-89fd-2cb3ff6bf4f7 is being executed.
INFO:sagemaker:Query 37b3fff8-a838-483f-89fd-2cb3ff6bf4f7 successfully executed.


,date,store_nbr,sales,oil,onpromotion,is_holiday,city,state,cluster,year,...,month_sin,day_cos,day_sin,dow_cos,dow_sin,sales_record_id,event_time,write_time,api_invocation_time,is_deleted
0,2013-01-01,1,0.0,93.14,0,1,18,12,13,2013,...,0.5,0.97953,0.201299,0.62349,0.781831,2013-01-01:1,1.728879e+09,2024-10-14 04:11:58.235,2024-10-14 04:06:44.000,False
1,2013-01-01,2,0.0,93.14,0,1,18,12,13,2013,...,0.5,0.97953,0.201299,0.62349,0.781831,2013-01-01:2,1.728879e+09,2024-10-14 04:11:58.188,2024-10-14 04:06:45.000,False
2,2013-01-01,3,0.0,93.14,0,1,18,12,8,2013,...,0.5,0.97953,0.201299,0.62349,0.781831,2013-01-01:3,1.728879e+09,2024-10-14 04:11:58.119,2024-10-14 04:06:45.000,False
3,2013-01-01,4,0.0,93.14,0,1,18,12,9,2013,...,0.5,0.97953,0.201299,0.62349,0.781831,2013-01-01:4,1.728879e+09,2024-10-14 04:11:57.992,2024-10-14 04:06:45.000,False
4,2013-01-01,5,0.0,93.14,0,1,21,14,4,2013,...,0.5,0.97953,0.201299,0.62349,0.781831,2013-01-01:5,1.728879e+09,2024-10-14 04:11:58.307,2024-10-14 04:06:45.000,False


In [186]:
# sales_features_store_df.to_csv("input_data.csv")
# !aws s3 cp "input_data.csv" $input_data_path

upload: ./input_data.csv to s3://sagemaker-us-east-1-342408968837/store-sales-forecasting/pipelines/input/input_data.csv


In [112]:
model_package_group_name = "custom-model-package-group"

model_packages = sm_client.list_model_packages(
    ModelPackageGroupName=model_package_group_name, SortBy="CreationTime", SortOrder="Descending")

if len(model_packages['ModelPackageSummaryList']) > 0:
    model_package = sm_client.describe_model_package(ModelPackageName=model_packages['ModelPackageSummaryList'][0]["ModelPackageArn"])
    model_data = model_package["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]
    registered_model_name = model_data.rsplit("/", 3)[1]
    print(model_name)
else:
    print("No packages") 

pipelines-oxp8zu7d2mok-CustomModelTrain-iufzybiEQR


In [113]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)
input_data = ParameterString(
    name="InputData",
    default_value=input_data_path,
)
batch_data = ParameterString(
    name="BatchData",
    default_value=output_data_path,
)
rmse_threshold = ParameterFloat(name="RmseThreshold", default_value=0.65)

In [114]:
%%writefile preprocessing.py
import json
import argparse
import os
import requests
import tempfile

import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def get_store_features(row):
    return [
      row["sales"], 
      row["oil"], 
      row["onpromotion"],
      row["is_holiday"], 
      row["hash_0"], 
      row["hash_1"], 
      row["hash_2"], 
      row["hash_3"], 
      row["hash_4"], 
      row["hash_5"], 
      row["hash_6"], 
      row["hash_7"], 
      row["hash_8"], 
      row["hash_9"], 
      row["month_cos"],
      row["month_sin"],
      # row["day_cos"],
      # row["day_sin"],
      row["dow_cos"],
      row["dow_sin"]
]

def generate_windows(data, input_seq_length, target_seq_length, stride):
    windows = []
    targets = []
    num_days = data.shape[1]
    
    for i in range(0, num_days, stride):
        if (i+input_seq_length+target_seq_length) <= num_days:
            input_window_end = i + input_seq_length
            target_window_end = input_window_end + target_seq_length
            
            input_window = data[:, i:input_window_end, :]
            target_window = data[:, input_window_end:target_window_end, 0]
            
            windows.append(input_window)
            targets.append(target_window)
            
    return np.array(windows), np.array(targets)



if __name__ == "__main__":
    base_dir = "/opt/ml/processing"

    df = pd.read_csv(f"{base_dir}/input/input_data.csv", index_col=0)
    
    # Apply to the whole dataframe
    df["features"] = df.apply(get_store_features, axis=1)
    num_continuous_features = 3
    
    drop_columns = [col for col in df.columns if col not in ["date", "store_nbr", "features"]]
    df.drop(columns=drop_columns, inplace=True)
    
    # Pivot the data to be in the format (store number, date, features)
    df_pivoted = df.pivot(index="store_nbr", columns="date", values="features")
    
    # Convert the data to an array
    stacked_df = np.array(df_pivoted.values.tolist())
    
    # Split the data into test/train/val sets with a 80/10/10 split
    n = stacked_df.shape[1]
    train_data = stacked_df[:, :int(n*0.8), :]
    test_data = stacked_df[:, int(n*0.8):int(n*0.9), :]
    val_data = stacked_df[:, int(n*0.9):, :]
    
    # Get the mean and standard deviation for normalization
    scaler = StandardScaler()

    # Flatten the first 2 dimensions into (stores*instances, features)
    train_data_2d = train_data.reshape(-1, train_data.shape[2])
    test_data_2d = test_data.reshape(-1, test_data.shape[2])
    val_data_2d = val_data.reshape(-1, val_data.shape[2])

    # Scale just the continuous features
    train_data_2d[:, :num_continuous_features] = scaler.fit_transform(train_data_2d[:, :num_continuous_features])
    test_data_2d[:, :num_continuous_features] = scaler.transform(test_data_2d[:, :num_continuous_features])
    val_data_2d[:, :num_continuous_features] = scaler.transform(val_data_2d[:, :num_continuous_features])

    # Add Gaussian noise to the continuous features
    train_data_2d[:, :num_continuous_features] = train_data_2d[:, :num_continuous_features] + np.random.normal(0, 0.3, train_data_2d[:, :num_continuous_features].shape)
    test_data_2d[:, :num_continuous_features] = test_data_2d[:, :num_continuous_features] + np.random.normal(0, 0.3, test_data_2d[:, :num_continuous_features].shape)
    val_data_2d[:, :num_continuous_features] = val_data_2d[:, :num_continuous_features] + np.random.normal(0, 0.3, val_data_2d[:, :num_continuous_features].shape)

    # Reshape the data back to its original dimensions
    train_data = train_data_2d.reshape(train_data.shape)
    test_data = test_data_2d.reshape(test_data.shape)
    val_data = val_data_2d.reshape(val_data.shape)
    
    # Generate windows for train/test/val sets
    input_seq_length = 7
    target_seq_length = 1
    stride = 1

    train_inputs, train_targets = generate_windows(train_data, input_seq_length, target_seq_length, stride)
    print(f"Train inputs shape: {train_inputs.shape}")
    print(f"Train targets shape: {train_targets.shape}")

    test_inputs, test_targets = generate_windows(test_data, input_seq_length, target_seq_length, stride)
    print(f"Test inputs shape: {test_inputs.shape}")
    print(f"Test targets shape: {test_targets.shape}")

    val_inputs, val_targets = generate_windows(val_data, input_seq_length, target_seq_length, stride)
    print(f"Validation inputs shape: {val_inputs.shape}")
    print(f"Validation inputs shape: {val_targets.shape}")
    
    # Save data splits
    np.save(f"{base_dir}/train/train_inputs.npy", train_inputs)
    np.save(f"{base_dir}/train/train_targets.npy", train_targets)

    np.save(f"{base_dir}/test/test_inputs.npy", test_inputs)
    np.save(f"{base_dir}/test/test_targets.npy", test_targets)

    np.save(f"{base_dir}/validation/val_inputs.npy", val_inputs)
    np.save(f"{base_dir}/validation/val_targets.npy", val_targets)

    with open(f"{base_dir}/transform-input/validation_data.ndjson", "w") as f:
        for i, window in enumerate(val_inputs):
            instance = {"input_1": window.tolist()}
            json_line = json.dumps(instance)
            if i < len(val_inputs) - 1:
                f.write(json_line + "\n")
            else:
                f.write(json_line)



Overwriting preprocessing.py


In [115]:
from sagemaker.sklearn.processing import SKLearnProcessor


framework_version = "1.2-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type="ml.m5.xlarge",
    instance_count=processing_instance_count,
    base_job_name="sklearn-custom-model-process",
    role=role,
    sagemaker_session=pipeline_session,
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [116]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
        ProcessingOutput(output_name="transform-input", source="/opt/ml/processing/transform-input"),
    ],
    code="preprocessing.py",
)

step_process = ProcessingStep(name="CustomModelProcess", step_args=processor_args)

In [117]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.tensorflow import TensorFlow

# Define regex patterns for capturing training metrics
metric_definitions=[
    {'Name': 'loss', 'Regex': "loss: ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'root_mean_squared_error', 'Regex': "root_mean_squared_error: ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'mean_absolute_error', 'Regex': "mean_absolute_error: ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'val_loss', 'Regex': "val_loss: ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'val_root_mean_squared_error', 'Regex': "val_root_mean_squared_error: ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'val_mean_absolute_error', 'Regex': "val_mean_absolute_error: ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'epoch', 'Regex': "Epoch ([0-9]+(.|e\-)[0-9]+),?"}]

image_uri = sagemaker.image_uris.retrieve(
    framework='tensorflow',
    region=region,
    version='2.6.0',
    image_scope='training',
    instance_type='ml.m5.xlarge'
)

# Define an estimator using the TensorFlow container and custom training logic
model_path = f"s3://{bucket}/CustomModelTrain"

custom_model_train = Estimator(
    entry_point='train.py',
    image_uri=image_uri,
    instance_type='ml.m5.xlarge',
    instance_count=1,
    output_path=model_path,
    role=role,
    sagemaker_session=pipeline_session,
    hyperparameters={
        'batch_size': 10,
        'epochs': 50,
        'learning_rate': 0.002,
        'l2_regularization': 0.004,
        'dropout': 0.2
    },
    metric_definitions=metric_definitions
)

train_args = custom_model_train.fit(
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="application/x-npy",
        ),
        "test": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            content_type="application/x-npy",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="application/x-npy",
        ),
    }
)


INFO:sagemaker.image_uris:Defaulting to only available Python version: py38


In [118]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="CustomModelTrain",
    step_args=train_args,
)

In [119]:
from sagemaker.model import Model
custom_model = Model(
    image_uri='763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.6-cpu',
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
)

In [120]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep

step_create_model = ModelStep(
    name="CustomModelCreateEvalModel",
    step_args=custom_model.create(instance_type="ml.m5.xlarge"),
)

In [15]:
output_data_path

's3://sagemaker-us-east-1-342408968837/store-sales-forecasting/pipelines/output/'

In [121]:
from sagemaker.transformer import Transformer
transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    strategy="MultiRecord",
    assemble_with="Line",
    output_path=f"{output_data_path}transform-results",
    accept="application/jsonlines"
)
    

In [122]:
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep
transform_input = TransformInput(
    data=step_process.properties.ProcessingOutputConfig.Outputs["transform-input"].S3Output.S3Uri, 
    split_type="Line",
    content_type="application/jsonlines"
)

step_transform_eval = TransformStep(
    name="CustomModelBatchTransform", transformer=transformer, inputs=transform_input
)

In [256]:
# !aws s3 cp "s3://sagemaker-us-east-1-342408968837/CustomModelTrain/pipelines-cjp3wuuvp642-CustomModelTrain-EWL2tdQjZq/output/model.tar.gz" "./"

download: s3://sagemaker-us-east-1-342408968837/CustomModelTrain/pipelines-cjp3wuuvp642-CustomModelTrain-EWL2tdQjZq/output/model.tar.gz to ./model.tar.gz


In [123]:
%%writefile evaluation.py
import os
import json
import pathlib
import numpy as np

from sklearn.metrics import mean_absolute_error, mean_squared_error


if __name__ == "__main__":
    
    base_dir = "/opt/ml/processing"
    print(os.getcwd())
    
    val_targets = np.load(os.path.join(f"{base_dir}/validation", "val_targets.npy"))
    print(val_targets.shape)
    
    with open(f"{base_dir}/transform-results/validation_data.ndjson.out", "r") as f:
        predictions = []
        for line in f:
            obj = json.loads(line.strip())
            predictions.extend(obj["predictions"])

    predictions_array = np.array(predictions)
    print(predictions_array.shape)
    
    targets_flat = val_targets.flatten()
    predictions_flat = predictions_array.flatten()

    rmse = mean_squared_error(targets_flat, predictions_flat, squared=False)
    mae = mean_absolute_error(targets_flat, predictions_flat)
    std = np.std(targets_flat - predictions_flat)
    print(f"RMSE: {rmse} MAE: {mae}")

    report_dict = {
        "regression_metrics": {
            "rmse": {"value": rmse, "standard_deviation": std},
            "mae": {"value": mae, "standard_deviation": std}
        },
    }

    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))

Overwriting evaluation.py


In [124]:
# from sagemaker.processing import ScriptProcessor

from sagemaker.sklearn.processing import SKLearnProcessor


framework_version = "1.2-1"

sklearn_eval_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type="ml.m5.xlarge",
    instance_count=processing_instance_count,
    base_job_name="sklearn-custom-eval-process",
    role=role,
    sagemaker_session=pipeline_session,
)

eval_args = sklearn_eval_processor.run(
    inputs=[
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri,
            destination="/opt/ml/processing/validation",
        ),
        ProcessingInput(
            source=step_transform_eval.properties.TransformOutput.S3OutputPath,
            destination="/opt/ml/processing/transform-results",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="evaluation.py",
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [125]:
from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)
step_eval = ProcessingStep(
    name="CustomModelEval",
    step_args=eval_args,
    property_files=[evaluation_report],
)

In [126]:
step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]


's3://sagemaker-us-east-1-342408968837/sklearn-custom-eval-process-2024-10-18-15-54-53-578/output/evaluation'

In [127]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

register_args = custom_model.register(
    # content_types=["text/csv"],
    # response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)
step_register = ModelStep(name="CustomModelRegisterModel", step_args=register_args)

In [128]:
from sagemaker.transformer import Transformer
new_forecast_transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    strategy="MultiRecord",
    assemble_with="Line",
    output_path=f"{output_data_path}sales-forecast",
    accept="application/jsonlines"
)

old_forecast_transformer = Transformer(
    model_name=registered_model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    strategy="MultiRecord",
    assemble_with="Line",
    output_path=f"{output_data_path}sales-forecast",
    accept="application/jsonlines"
)

In [129]:
transform_input_forecast = TransformInput(
    data=step_process.properties.ProcessingOutputConfig.Outputs["transform-input"].S3Output.S3Uri, 
    split_type="Line"
)

step_transform_forecast_new = TransformStep(
    name="CustomModelBatchForecastNew", transformer=new_forecast_transformer, inputs=transform_input_forecast
)

step_transform_forecast_existing = TransformStep(
    name="CustomModelBatchForecastExisting", transformer=old_forecast_transformer, inputs=transform_input_forecast
)


In [132]:
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import Join

step_fail = FailStep(
    name="CustomModelRMSEFail",
    error_message=Join(on=" ", values=["Execution failed due to RMSE >", rmse_threshold, ". Using previous successful deployment for forecasting"]),
)

In [133]:
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet


cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="regression_metrics.rmse.value",
    ),
    right=rmse_threshold,
)

step_cond = ConditionStep(
    name="CustomModelRMSECond",
    conditions=[cond_lte],
    if_steps=[step_register, step_transform_forecast_new],
    else_steps=[step_fail, step_transform_forecast_existing],
)

In [134]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = f"CustomModelPipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        instance_type,
        model_approval_status,
        input_data,
        batch_data,
        rmse_threshold,
    ],
    steps=[step_process, step_train, step_create_model, step_transform_eval, step_eval, step_cond],
    #steps=[step_process, step_train, step_create_model, step_eval],
    #steps=[step_process, step_train, step_create_model, step_transform_eval, step_eval],
    #steps=[step_process, step_train, step_eval, step_cond],
)

In [ ]:
import json

definition = json.loads(pipeline.definition())
definition

In [135]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:342408968837:pipeline/CustomModelPipeline',
 'ResponseMetadata': {'RequestId': '1f05094c-e3cf-414c-8129-9f0c7bb236b3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1f05094c-e3cf-414c-8129-9f0c7bb236b3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '87',
   'date': 'Fri, 18 Oct 2024 16:07:56 GMT'},
  'RetryAttempts': 0}}

In [136]:
execution = pipeline.start()
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:342408968837:pipeline/CustomModelPipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:342408968837:pipeline/CustomModelPipeline/execution/for3hm0bpcrb',
 'PipelineExecutionDisplayName': 'execution-1729267678935',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2024, 10, 18, 16, 7, 58, 868000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 10, 18, 16, 7, 58, 868000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:342408968837:user-profile/d-2cr7fbmrqyrg/jlawton',
  'UserProfileName': 'jlawton',
  'DomainId': 'd-2cr7fbmrqyrg',
  'IamIdentity': {'Arn': 'arn:aws:sts::342408968837:assumed-role/LabRole/SageMaker',
   'PrincipalId': 'AROAU7OJKHKCWCCLLHI6O:SageMaker'}},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:342408968837:user-profile/d-2cr7fbmrqyrg/jlawton',
  'UserProfileName': 'jlawton',
  'DomainId': 'd-2cr7fbmrqyrg',
  'IamIdenti

In [ ]:
execution.wait()